In [10]:
# pip install requests

In [11]:
import os
import requests
import pandas as pd

API_KEY = "3fe140f3649087372bbe9eb665dff7c4"   

if not API_KEY or API_KEY.strip() == "":
    raise SystemExit("Please paste your FRED API key in API_KEY before running further.")

In [ ]:
BASE = "https://api.stlouisfed.org/fred/series/observations"

# Weekly constant maturity yields (H.15, weekly ending Friday)
SERIES = {
    "WGS3MO": "y_3m",   # 3-Month
    "WGS1YR": "y_1y",   # 1-Year
    "WGS2YR": "y_2y",   # 2-Year
    "WGS5YR": "y_5y",   # 5-Year
    "WGS10YR":"y_10y",  # 10-Year
    "WGS30YR":"y_30y"   # 30-Year  
}

OBS_START = "1970-01-01" 

def fetch_fred_series(series_id: str, series_name: str, obs_start: str = OBS_START) -> pd.DataFrame:
    """Download one FRED series (weekly) and return a 2-col DataFrame indexed by date."""
    params = {
        "series_id": series_id,
        "file_type": "json",
        "api_key": API_KEY,
        "observation_start": obs_start
    }
    r = requests.get(BASE, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    obs = js.get("observations", [])
    df = pd.DataFrame(obs)
    if df.empty:
        raise ValueError(f"No data for series {series_id}")
    df["date"] = pd.to_datetime(df["date"])
    df["value"] = pd.to_numeric(df["value"], errors="coerce")
    return df[["date", "value"]].rename(columns={"value": series_name}).set_index("date")

In [ ]:
dfs = []
for sid, colname in SERIES.items():
    print(f"Downloading {sid} → {colname} ...")
    df_i = fetch_fred_series(sid, colname)
    dfs.append(df_i)

yields_w = pd.concat(dfs, axis=1).sort_index()

yields_w["spr_10y_3m"] = yields_w["y_10y"] - yields_w["y_3m"]
yields_w["spr_10y_2y"] = yields_w["y_10y"] - yields_w["y_2y"]

# Quick QA
print("\nShape:", yields_w.shape)
print("Date range:", yields_w.index.min().date(), "→", yields_w.index.max().date())
print("\nMissing values per column:")
print(yields_w.isna().sum())
yields_w.tail(3)


Shape: (2915, 8)
Date range: 1970-01-02 → 2025-11-07

Missing values per column:
y_3m          609
y_1y            0
y_2y          335
y_5y            0
y_10y           0
y_30y         372
spr_10y_3m    609
spr_10y_2y    335
dtype: int64


,y_3m,y_1y,y_2y,y_5y,y_10y,y_30y,spr_10y_3m,spr_10y_2y
date,,,,,,,,
2025-10-24,3.95,3.57,3.46,3.58,4.00,4.57,0.05,0.54
2025-10-31,3.90,3.66,3.55,3.67,4.06,4.61,0.16,0.51
2025-11-07,3.95,3.67,3.59,3.71,4.12,4.70,0.17,0.53


In [18]:
os.makedirs("Yeild-Curve-Forecasting/Data/raw", exist_ok=True)
out_path = "Yeild-Curve-Forecasting/Data/raw/treasury_yields_weekly.csv"
yields_w.to_csv(out_path, float_format="%.4f")
out_path

'Yeild-Curve-Forecasting/Data/raw/treasury_yields_weekly.csv'

In [ ]:
df = pd.read_csv("Yeild-Curve-Forecasting/Data/raw/treasury_yields_weekly.csv", parse_dates=["date"], index_col="date")

# last few rows of key columns
display(df.tail(5)[["y_3m","y_10y","spr_10y_3m"]])

# How often was the curve inverted historically? (10Y - 3M < 0)
inv_share = (df["spr_10y_3m"] < 0).mean()
print(f"Inversion share of weeks: {inv_share:.2%}")

,y_3m,y_10y,spr_10y_3m
date,,,
2025-10-10,4.02,4.13,0.11
2025-10-17,4.02,4.02,0.00
2025-10-24,3.95,4.00,0.05
2025-10-31,3.90,4.06,0.16
2025-11-07,3.95,4.12,0.17


Inversion share of weeks: 8.92%


In [ ]:
import matplotlib.pyplot as plt

df["spr_10y_3m"].dropna().plot(figsize=(10,3), title="10Y - 3M spread (weekly)")
plt.axhline(0, linestyle="--")
plt.show()